In [ ]:
import numpy as np 
%matplotlib qt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import ipywidgets as widgets
from ipyfilechooser import FileChooser
import pandas as pd
import os
import warnings
from itertools import zip_longest
warnings.filterwarnings('ignore')

## Preamble: Useful functions (to run)

In [ ]:
def clean_df(df): 
    """Makes sure that df elements are floats and replaces
    inf with nan for easier data manipulation."""
    a = df.astype(float).replace([-np.inf,+np.inf],np.nan)
    return a

# Introduction
This notebook allows to visualise and manipulate Brillouin Spectroscopy maps (in terms of frequency shift, $\nu_B$ and width, $\Delta_B$) acquired and processed using the light machinery set-up (https://lightmachinery.com/). In brief, the "Scan_Parameters_and_Analysis.csv" file is first imported and manipulated so that the Brillouin map ($\nu_B$ and $\Delta_B$) can be shown as a color map with appropriate $x$ and $y$ coordinates.
Then, $\nu_B$ and $\Delta_B$  profiles are shown along both $x$ and $y$ as line plots. Finally, the possibility to isolate the background from the objects of interest (e.g., cells, spheroids etc) is given using a simple threshold or by using a mask created in python / Fiji. 
 <br> 
The notebook has been made interactive via widgets to be more user friendly. A step by step workflow is summarised below: 

### 1) Run cell below to upload the  "Scan_Parameters_and_Analysis.csv" file 

In [ ]:
fc = FileChooser()
display(fc)

The cell below will display the acquired map and prints its dimensions. If you think the map looks suspicious (i.e., contains outliers), select "Yes" below and then choose a range of frequencies you want to restric your data to based on its 2D histogram. If the map looks ok, then the origianl data will be used!

In [ ]:
#Read data and create sub-data frame with X, Y and Shift; add columns in um (from mm)
data = pd.read_csv(fc.selected)
datasub = data[[" X (mm)", " Y (mm)", " Shift (GHz)"," FWHM (GHz)"]]
datasub["X (um)"]=datasub.iloc[:,0]* 1000.0
datasub["Y (um)"]=datasub.iloc[:,1]* 1000.0
#Pivot sub-data frame so that it can be plotted as a map with correct x and y coordinates
piv=datasub.pivot(index = "Y (um)",columns="X (um)")
piv=clean_df(piv)
print("Pixels in y: %d"%(piv[" Shift (GHz)"].shape[0]))
print("Pixels in x: %d"%(piv[" Shift (GHz)"].shape[1]))
fig,ax = plt.subplots(1,2, figsize=(8,4))
ax[0].imshow(piv[" Shift (GHz)"],cmap="viridis",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
ax[1].imshow(piv[" FWHM (GHz)"],cmap="magma",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
#label axes and add colorbar
labels = ["Shift (GHz)","FWHM (GHz)"]
for i in ax:
    i.set_xlabel("x ($\mu$m)")
    i.set_ylabel("y ($\mu$m)")
    divider = make_axes_locatable(i)
    cax = divider.append_axes('right', size='8%', pad=0.05)
    plt.colorbar(mappable=i.get_images()[0],ax=i,cax=cax,fraction=0.05,label=labels[ax.tolist().index(i)])
fig.tight_layout()

### 2) Time for python and Fiji - get out of this notebook!
Now that you know the map's dimensions, it's time to run the python script "yellowcrop.py" in the "crop_brightfield" folder.
This crops the brightfield image from the light machinery software around the provided yellow square (ie the area acquired via Brillouin, too): 
1. Activate brillouimaps environment;
2. Run the script yellowcrop.py on the brightfield BMP image (python dir/yellocrop.py imagedir/image.BMP); dir is the directory where yellowcrop.py is and imagedir is the directory where image.BMP is. This saves the cropped image (in .tiff format) in the same directory as image.BMP;
3. Open Fiji to downsize the image to the Brillouin map's dimensions and create two masks, one for the object and one for the background:
    1. Open Fiji and load the image_cropped.tif
    2. Manually segment the feature of interest
    3. Create a mask (edit-> selection -> create mask)
    4. Downsize the image to the same pixel size of the Brillouin map (Image -> scale)
    5. Select non-interpolated pixels, ie white pixels (Image -> Adjust -> Threshold -> 255 to 255; Apply)
    6. Make binary image (Process -> Math -> divide by 255). The image will appear black because 0 and 1 are very close, but the object is retained and can be checked by image-> adjust ->        brightness/contrast.
    7. Save as tiff image (object.tiff)
    8. Invert object.tiff (edit->invert) and subtract 254 to have the opposite image (background.tiff). Save the inverted tiff image


### 3) Load the created mask, either for the object or background

In [ ]:
maskfc = FileChooser()
display(maskfc)

In [ ]:
object_mask_fiji = plt.imread(maskfc.selected)
fig,ax = plt.subplots(1,1, figsize=(4,4))
plt.imshow(object_mask_fiji,cmap='gray',interpolation='none')
plt.title("Mask")
plt.show()

### 4) Run cell below to display line profiles along x and  y

In [ ]:
#plot line profiles of the data along x and y
data_shift = piv[" Shift (GHz)"]
data_width = piv[" FWHM (GHz)"]
#Shift
mean_x_shift = data_shift.mean(axis=0)
std_x_shift = data_shift.std(axis=0)
mean_y_shift = data_shift.mean(axis=1)
std_y_shift = data_shift.std(axis=1)
#FWHM
mean_x_width = data_width.mean(axis=0)
std_x_width = data_width.std(axis=0)
mean_y_width = data_width.mean(axis=1)
std_y_width = data_width.std(axis=1)

fig,ax = plt.subplots(2,2, figsize=(5,5))
ax[0,0].plot(data_shift.columns,mean_x_shift)
ax[0,0].fill_between(data_shift.columns,mean_x_shift+std_x_shift/2.0,mean_x_shift-std_x_shift/2.0,alpha=0.3)
ax[0,0].set_xlabel("x ($\mu$m)")
ax[0,0].set_ylabel("Shift (GHz)")
ax[0,1].plot(mean_y_shift,data_shift.index,color="C1")
ax[0,1].fill_betweenx(data_shift.index,mean_y_shift+std_y_shift/2.0,mean_y_shift-std_y_shift/2.0,alpha=0.3,color="C1")
ax[0,1].set_xlabel("Shift (GHz)")
ax[0,1].set_ylabel("y ($\mu$m)")

ax[1,0].plot(data_width.columns,mean_x_width,color="C2")
ax[1,0].fill_between(data_width.columns,mean_x_width+std_x_width/2.0,mean_x_width-std_x_width/2.0,alpha=0.3,color="C2")
ax[1,0].set_xlabel("x ($\mu$m)")
ax[1,0].set_ylabel("FWHM (GHz)")
ax[1,1].plot(mean_y_width,data_width.index,color="C3")
ax[1,1].fill_betweenx(data_width.index,mean_y_width+std_y_width/2.0,mean_y_width-std_y_width/2.0,alpha=0.3,color="C3")
ax[1,1].set_xlabel("FWHM (GHz)")
ax[1,1].set_ylabel("y ($\mu$m)")
fig.tight_layout()

### 5) Run cell below in order to interactively threshold the Brillouin map <br> - Shift
Briefly, a simple threshold based on the 2D histogram of the flattened map is used for discriminating between background and object of interest. Plots are automatically updated and the average for both background and object is shown as the threshold is changed. <bk>
You can copy and paste those values in Prism in order to run statistics.

In [ ]:
%matplotlib inline
def threshold_image_shift(threshold=6.20):
    data_shift = piv[" Shift (GHz)"]
    back = data_shift[data_shift<threshold]
    sph = data_shift[data_shift>threshold]
    fig,axs= plt.subplots(1,4,figsize=(15,3))
    axs[0].hist(np.array(data_shift).flatten(),bins="auto",color='k')
    axs[0].axvline(threshold,color='red')
    im1=axs[1].imshow(data_shift,cmap="viridis",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
    im2=axs[2].imshow(back,cmap="viridis",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
    im3=axs[3].imshow(sph,cmap="viridis",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
    all = [im1,im2,im3]
    titles = ["2D histogram", "Full map", "Background", "Object"]
    for i,ax in enumerate(axs):
        ax.set_title(titles[i])
        ax.grid(False)
        if i == 0:
            ax.set_ylabel("Counts")
            ax.set_xlabel("Shift (GHz)")
            #skip color map for histogram
            continue
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='8%', pad=0.05)
        ax.set_xlabel("x $(\mu m)$")
        ax.set_ylabel("y $(\mu m)$")
        plt.colorbar(mappable=all[i-1],ax=axs[i],cax=cax,fraction=0.05,label="Shift (GHz)")
    fig.tight_layout()
plt.close()
threshold_image_interactive_shift= widgets.interactive(threshold_image_shift,threshold=(5,7,0.01))
display(threshold_image_interactive_shift)                             

### 6) Run cell below in order to compute the mean of the background and object <bk> in terms of shift
You can then copy and paste those values in any software of preference (e.g., Prism) for statistical tests and furhter plotting!

In [ ]:
data_shift = piv[" Shift (GHz)"]
threshold_shift = threshold_image_interactive_shift.kwargs["threshold"]
mean_back_th_shift = np.nanmean(data_shift[data_shift<threshold_image_interactive_shift.kwargs["threshold"]])
sd_back_th_shift = np.nanstd(data_shift[data_shift<threshold_image_interactive_shift.kwargs["threshold"]])
mean_object_th_shift = np.nanmean(data_shift[data_shift>threshold_image_interactive_shift.kwargs["threshold"]])
sd_object_th_shift = np.nanstd(data_shift[data_shift>threshold_image_interactive_shift.kwargs["threshold"]])
print("Threshold value: {threshold: .2f}".format(threshold=threshold_shift))
print("Object's Average Brillouin Shift p/m 1SD (GHz):{mean_object_th_shift: .3f} p/m {sd_object_th_shift:.3f}".format(mean_object_th_shift=mean_object_th_shift, 
                                                                                                          sd_object_th_shift=sd_object_th_shift))
print("Background's Average Brillouin Shift p/m 1SD (GHz):{mean_back_th_shift: .3f} p/m {sd_back_th_shift:.3f}".format(mean_back_th_shift=mean_back_th_shift,
                                                                                                          sd_back_th_shift=sd_back_th_shift))

### 7) Run cell below in order to interactively threshold the Brillouin map <br> - Width
Briefly, a simple threshold based on the 2D histogram of the flattened map is used for discriminating between background and object of interest. Plots are automatically updated and the average for both background and object is shown as the threshold is changed. <bk>
You can copy and paste those values in Prism in order to run statistics.

In [ ]:
%matplotlib inline
def threshold_image_width(threshold=0.9):
    data_fwhm = piv[" FWHM (GHz)"]
    back = data_fwhm[data_fwhm<threshold]
    sph = data_fwhm[data_fwhm>threshold]
    fig,axs= plt.subplots(1,4,figsize=(15,3))
    axs[0].hist(np.array(data_fwhm).flatten(),bins="auto",color='k')
    axs[0].axvline(threshold,color='red')
    im1=axs[1].imshow(data_fwhm,cmap="magma",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
    im2=axs[2].imshow(back,cmap="magma",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
    im3=axs[3].imshow(sph,cmap="magma",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),min(datasub["Y (um)"]),max(datasub["Y (um)"])])
    all = [im1,im2,im3]
    titles = ["2D histogram", "Full map", "Background", "Object"]
    for i,ax in enumerate(axs):
        ax.set_title(titles[i])
        ax.grid(False)
        if i == 0:
            ax.set_ylabel("Counts")
            ax.set_xlabel(" FWHM(GHz)")
            #skip color map for histogram
            continue
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='8%', pad=0.05)
        ax.set_xlabel("x $(\mu m)$")
        ax.set_ylabel("y $(\mu m)$")
        plt.colorbar(mappable=all[i-1],ax=axs[i],cax=cax,fraction=0.05,label="FWHM (GHz)")
    fig.tight_layout()
plt.close()
threshold_image_interactive_width= widgets.interactive(threshold_image_width,threshold=(0,2,0.01))
display(threshold_image_interactive_width) 

### 8) Run cell below in order to compute the mean of the background and object <bk> in terms of width
You can then copy and paste those values in any software of preference (e.g., Prism) for statistical tests and furhter plotting!

In [ ]:
data_width = piv[" FWHM (GHz)"]
threshold_width = threshold_image_interactive_width.kwargs["threshold"]
mean_back_th_width = np.nanmean(data_width[data_width<threshold_image_interactive_width.kwargs["threshold"]])
sd_back_th_width = np.nanstd(data_width[data_width<threshold_image_interactive_width.kwargs["threshold"]])
mean_object_th_width = np.nanmean(data_width[data_width>threshold_image_interactive_width.kwargs["threshold"]])
sd_object_th_width = np.nanstd(data_width[data_width>threshold_image_interactive_width.kwargs["threshold"]])
print("Threshold value: {threshold: .2f}".format(threshold=threshold_width))
print("Object's Average Brillouin Width p/m 1SD (GHz):{mean_object_th_width: .3f} p/m {sd_object_th_width:.3f}".format(mean_object_th_width=mean_object_th_width, 
                                                                                                          sd_object_th_width=sd_object_th_width))
print("Background's Average Brillouin Width p/m 1SD (GHz):{mean_back_th_width: .3f} p/m {sd_back_th_width:.3f}".format(mean_back_th_width=mean_back_th_width,
                                                                                                          sd_back_th_width=sd_back_th_width))

### 9) Run cell below in order to mask the brillouin map based on the bf image mask <bk>



In [ ]:
#take only Brillouin map where mask is 1
data_shift = piv[" Shift (GHz)"]
data_width = piv[" FWHM (GHz)"]
segmented_shift = np.array(data_shift)*object_mask_fiji
segmented_width = np.array(data_width)*object_mask_fiji

fig,ax = plt.subplots(1,2, figsize=(8,4))
segmented_image_shift = ax[0].imshow(segmented_shift,cmap="hot",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),
                                                         min(datasub["Y (um)"]),max(datasub["Y (um)"])])
segmented_image_width = ax[1].imshow(segmented_width,cmap="hot",extent=[min(datasub["X (um)"]),max(datasub["X (um)"]),
                                                          min(datasub["Y (um)"]),max(datasub["Y (um)"])])
labels = ["Shift (GHz)","FWHM (GHz)"]
for i in ax:
    i.set_xlabel("x ($\mu$m)")
    i.set_ylabel("y ($\mu$m)")
    divider = make_axes_locatable(i)
    cax = divider.append_axes('right', size='8%', pad=0.05)
    plt.colorbar(mappable=i.get_images()[0],ax=i,cax=cax,fraction=0.05,label=labels[ax.tolist().index(i)])
fig.tight_layout()
#select only relevant Brillouin pixels
brillouin_object_shift = segmented_shift[segmented_shift>0]
mean_brillouin_object_shift = np.mean(brillouin_object_shift)
std_brillouin_object_shift = np.std(brillouin_object_shift)
brillouin_object_width = segmented_width[segmented_width>0]
mean_brillouin_object_width = np.mean(brillouin_object_width)
std_brillouin_object_width = np.std(brillouin_object_width)
print("Object's Average Brillouin Shift p/m 1SD (GHz):{mean_brillouin_object_shift: .3f} p/m {std_brillouin_object_shift:.3f}".format(mean_brillouin_object_shift=mean_brillouin_object_shift,
                                                                                                                                      std_brillouin_object_shift=std_brillouin_object_shift))
print("Object's Average Brillouin Width p/m 1SD (GHz):{mean_brillouin_object_width: .3f} p/m {std_brillouin_object_width:.3f}".format(mean_brillouin_object_width=mean_brillouin_object_width,
                                                                                                                                      std_brillouin_object_width=std_brillouin_object_width))

### 10) And... Run the two cells below if you want a .tsv file with individual values from object and background in terms of shift, width
The file can be opened in Excel, and used for further analysis!

In [ ]:
data = pd.read_csv(fc.selected)
##print(data)
shift = data[" Shift (GHz)"]
width = data[" FWHM (GHz)"]
shift = clean_df(shift)
width = clean_df(width)

fw_shift = shift[shift>threshold_image_interactive_shift.kwargs["threshold"]] #object
bk_shift = shift[shift<threshold_image_interactive_shift.kwargs["threshold"]] #background
fw_width = width[width>threshold_image_interactive_width.kwargs["threshold"]] #object
bk_width = width[width<threshold_image_interactive_width.kwargs["threshold"]] #background
#need to check definition of loss tangent - is the 4*pi factor correct?
fw_loss_tangent = (4 * np.pi) * (fw_width / fw_shift) 
bk_loss_tangent = (4 * np.pi) * (bk_width / bk_shift)
object_loss_tangent = (4 * np.pi) * (brillouin_object_width / brillouin_object_shift)

data = {"Background Shift (GHz)": bk_shift, "Object Shift (GHz)": fw_shift, "Masked object Shift (GHz)": pd.Series(brillouin_object_shift),
        "Background Width (GHz)": bk_width, "Object Width (GHz)": fw_width, "Masked object Width (GHz)": pd.Series(brillouin_object_width),
        "Background Loss Tangent": bk_loss_tangent, "Object Loss Tangent": fw_loss_tangent, "Masked object Loss Tangent": pd.Series(object_loss_tangent)}

df = pd.DataFrame(data=data)

samplenamew=widgets.Text(placeholder='Please enter the sample name',disabled=False)

namebox=widgets.HBox([widgets.Label(value="Sample Name:"), samplenamew])
display(namebox)

savedir=widgets.Text(
    placeholder='Please enter saving directory',
    disabled=False
)

savebox=widgets.HBox([widgets.Label(value="Saving directory:"), savedir])
display(savebox)


In [ ]:
fname=savedir.value+"/"+samplenamew.value+".tsv"
with open(fname, 'w') as f:
    f.write("Background Shift (GHz) \t Object Shift (GHz) \t Masked Object Shift (GHz) \t Background Width (GHz) \t Object Width (GHz) \t Masked Object Width (GHz)\t Background Loss Tangent \t Object Loss Tangent \t Masked Object Loss Tangent\n")
    for x in zip_longest(*[bk_shift, fw_shift, brillouin_object_shift, bk_width, fw_width, brillouin_object_width, bk_loss_tangent, fw_loss_tangent, object_loss_tangent]):
        f.write("{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}\t{7}\t{8}\n".format(*x))